# This notebook shows how to automatically load today's games, and then predict/tweet said forecast

In [1]:
#Step 1. Load in the data and dependencies. 
%matplotlib inline
import numpy as np
import NNBA
import prediction_stuff
import matplotlib.pyplot as plt
np.random.seed(11)  #For reproducibility. And because Brook Lopez is the best. 
#note there is continuation, so if you run it twice without resetting the kernel the restuls will be different. 

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:

#Create a model for predicting the spread outcome of a game.

model,scaler = NNBA.make_network('NBADATA.csv',sklearn=False,
                                          keras=True,normalize=True,
                                          spread=True)




keras NN goes here
Train on 3820 samples, validate on 955 samples
Epoch 1/20
3820/3820 [==============================] - 1s 213us/step - loss: 0.8514 - acc: 0.4887 - val_loss: 0.7863 - val_acc: 0.4848
Epoch 2/20
3820/3820 [==============================] - 0s 40us/step - loss: 0.7233 - acc: 0.6277 - val_loss: 0.6933 - val_acc: 0.6942
Epoch 3/20
3820/3820 [==============================] - 0s 39us/step - loss: 0.5546 - acc: 0.7715 - val_loss: 0.5090 - val_acc: 0.8021
Epoch 4/20
3820/3820 [==============================] - 0s 40us/step - loss: 0.4286 - acc: 0.8285 - val_loss: 0.4523 - val_acc: 0.8283
Epoch 5/20
3820/3820 [==============================] - 0s 40us/step - loss: 0.3791 - acc: 0.8560 - val_loss: 0.4178 - val_acc: 0.8335
Epoch 6/20
3820/3820 [==============================] - 0s 39us/step - loss: 0.3535 - acc: 0.8626 - val_loss: 0.4672 - val_acc: 0.8052
Epoch 7/20
3820/3820 [==============================] - 0s 41us/step - loss: 0.3434 - acc: 0.8709 - val_loss: 0.3896 - val_

This printed result means that when it scanned over NBADATA.csv, a file created using nba_py (documented in NNBA.py), after training over 3820 games and testing over 955 games, the nbapredictor neural network could correctly classify whether or not an nba game hit the over or under 86% of the time.

Note this is based on the box scores of existing games! The challenge now is to find a way to predict what the box score of a future NBA game will turn out to be. In the cells below, we use the season splits of the Cleveland Cavaliers and Golden State Warriors to predict the spread winner of each game of the 2018 NBA Finals. 

In the end, this model ends up successfully --or the highest % belonging to the true winner--  4 out of the 4 times! I'd still consult other sources for game 3 though. Granted a small sample size, this model shows promise when extended throughout the entire NBA playoffs, and we look forward to seeing what comes of it start the 2018-19 NBA season. 

Of course there are many flaws with this system as the box score of a game will almost never be the exact regular season splits of a team. But this serves as a good starting point in learning more about NBA game prediction and modelling. 


And for other types of outcomes, the over under and ML are still experimental but follow the same procedure. Neither are as coherently programmed as this one so this is the launching point of our project. 

In [ ]:
CLE = prediction_stuff.get_splits('CLE')
GSW = prediction_stuff.get_splits('GSW')
spread1 = 13
spread2 = 12.5
spread3 = -3.5
spread4 = -4.5
spreads = [spread1,spread2,spread3,spread4]

CLEGSW1 = prediction_stuff.ML_game_maker(CLE,GSW,spread1,scaler)
CLEGSW2 = prediction_stuff.ML_game_maker(CLE,GSW,spread2,scaler)
CLEGSW3 = prediction_stuff.ML_game_maker(GSW,CLE,spread3,scaler)
CLEGSW4 = prediction_stuff.ML_game_maker(GSW,CLE,spread4,scaler)

the_finals = [CLEGSW1,CLEGSW2,CLEGSW3,CLEGSW4]

for i,game in enumerate(the_finals):
    prediction =  nbapredictorML.predict(game)
    plt.figure()
    # Data to plot
    labels = 'Cavs', 'Warriors'
    sizes = prediction[0]
    colors = ['lightcoral', 'gold', 'green']
    explode = (0.1, 0.1)  # explode each slice
 
    # Plot
    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
 
    plt.axis('equal')
    plt.title("Who Wins Game "+ str(i+1) + "?")

In [ ]:
from datetime import datetime

In [ ]:
now = datetime.now()

In [ ]:
now.year

In [ ]:
matchups = prediction_stuff.get_todays_games(now.day,now.month,now.year)

In [24]:
matchups = prediction_stuff.get_todays_games(3,2,2017)
print('matchups',matchups)
teamsplits = []
teams = []
for game in matchups:
    for team in game:
        teamsplit = prediction_stuff.get_splits(team)  #this is one of the arrays, now pair it up like was done
        teamsplits.append(teamsplit)
        teams.append(team)
        
home_teams_names = teams[1::2]
away_teams_names = teams[0::2]
home_teams = teamsplits[1::2]
away_teams = teamsplits[0::2]

spread = -6.5  #the spread for the road team, aka inverted sign for home team. 
#Spread will be extracted for that game...
todays_matchups = []
for i in range(len(home_teams)):
    todays_matchups.append(prediction_stuff.spread_game_maker(away_teams[i],home_teams[i],spread,scaler))

for i, game in enumerate(todays_matchups):
    
 #   print(game)
    print(away_teams_names[i],' vs. ', home_teams_names[i])
    prediction = list(model.predict(game)[0])

    winner = prediction.index(max(prediction))
    print('PREDICTION:')
    print("")
    if winner == 0: 
        #road team won the spread/mL, once I get this all together. 
        print(away_teams_names[i], spread)
    elif winner == 1:
        print(home_teams_names[i], -spread)
    else:
        print('PUSH')

    print(model.predict(game))

    print("")

Old Teams: ['TOR', 'ORL', 'IND', 'BKN', 'MIN', 'DET', 'CHI', 'HOU', 'MEM', 'OKC', 'LAL', 'BOS', 'MIL', 'DEN', 'PHX', 'SAC', 'DAL', 'POR']
Fixed(?) Teams:  ['TOR', 'ORL', 'IND', 'BRK', 'MIN', 'DET', 'CHI', 'HOU', 'MEM', 'OKC', 'LAL', 'BOS', 'MIL', 'DEN', 'PHX', 'SAC', 'DAL', 'POR']
matchups [['TOR', 'ORL'], ['IND', 'BRK'], ['MIN', 'DET'], ['CHI', 'HOU'], ['MEM', 'OKC'], ['LAL', 'BOS'], ['MIL', 'DEN'], ['PHX', 'SAC'], ['DAL', 'POR']]
TOR  vs.  ORL
PREDICTION:

TOR -6.5
[[ 0.94717818  0.0383355   0.01448635]]
IND  vs.  BRK
PREDICTION:

BRK 6.5
[[ 0.43933612  0.52418017  0.03648377]]
MIN  vs.  DET
PREDICTION:

DET 6.5
[[ 0.16354455  0.81220871  0.02424671]]
CHI  vs.  HOU
PREDICTION:

HOU 6.5
[[ 0.0074306   0.98921275  0.00335665]]
MEM  vs.  OKC
PREDICTION:

OKC 6.5
[[ 0.07226858  0.90879327  0.01893826]]
LAL  vs.  BOS
PREDICTION:

BOS 6.5
[[ 0.05241594  0.92376649  0.02381758]]
MIL  vs.  DEN
PREDICTION:

DEN 6.5
[[ 0.40418908  0.57218218  0.02362883]]
PHX  vs.  SAC
PREDICTION:

SAC 6.5
[[ 

In [18]:
prediction = list(prediction)

In [ ]:
new_teams = []
for team in teams:
    if team == 'BKN':
        new_teams.append('BRK')
    else:
        new_teams.append(team)

In [19]:
prediction.index(max(prediction))

0